In [1]:
import numpy as np
import pandas as pd
import pandas_ta as ta

import plotly.graph_objects as go

In [2]:
df = pd.read_csv('../dataset/dukascopy/EURUSD_Candlestick_1_Hour_BID_01.05.2023-18.05.2024.csv')

In [3]:
df.shape

(9216, 6)

In [4]:
df.tail()

,Gmt time,Open,High,Low,Close,Volume
9211,18.05.2024 19:00:00.000,1.08674,1.08674,1.08674,1.08674,0.0
9212,18.05.2024 20:00:00.000,1.08674,1.08674,1.08674,1.08674,0.0
9213,18.05.2024 21:00:00.000,1.08674,1.08674,1.08674,1.08674,0.0
9214,18.05.2024 22:00:00.000,1.08674,1.08674,1.08674,1.08674,0.0
9215,18.05.2024 23:00:00.000,1.08674,1.08674,1.08674,1.08674,0.0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9216 entries, 0 to 9215
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Gmt time  9216 non-null   object 
 1   Open      9216 non-null   float64
 2   High      9216 non-null   float64
 3   Low       9216 non-null   float64
 4   Close     9216 non-null   float64
 5   Volume    9216 non-null   float64
dtypes: float64(5), object(1)
memory usage: 432.1+ KB


## Filter holidays

**Volume is an indicator of liquity, also shows in which candle there was no trade**
<br/>
**In 1D timeframe, volume = 0 means day off or weekend.**

In [7]:
# df[['Date', 'Time']] = df['Gmt time'].str.split(' ', expand=True)
df['Date'] = df['Gmt time'].str.split(' ', expand=True)[0]


In [8]:
df.head(2)

,Gmt time,Open,High,Low,Close,Volume,Date
0,01.05.2023 00:00:00.000,1.10106,1.10143,1.10045,1.10055,8127.1401,01.05.2023
1,01.05.2023 01:00:00.000,1.10055,1.10084,1.10020,1.10040,7000.5898,01.05.2023


In [9]:
holidays =  (df[df['Volume'] == 0].groupby('Date').count()['Close'] >= 24).index
df_ops =  df[~ df['Date'].isin(holidays)]
print(df_ops.shape)
df_ops[df_ops['Volume'] == 0]

(5232, 7)


,Gmt time,Open,High,Low,Close,Volume,Date


In [10]:
df_ops.head()

,Gmt time,Open,High,Low,Close,Volume,Date
0,01.05.2023 00:00:00.000,1.10106,1.10143,1.10045,1.10055,8127.1401,01.05.2023
1,01.05.2023 01:00:00.000,1.10055,1.10084,1.10020,1.10040,7000.5898,01.05.2023
2,01.05.2023 02:00:00.000,1.10037,1.10081,1.10007,1.10081,3727.6499,01.05.2023
3,01.05.2023 03:00:00.000,1.10081,1.10140,1.10053,1.10140,4344.2002,01.05.2023
4,01.05.2023 04:00:00.000,1.10143,1.10143,1.10032,1.10062,4607.4600,01.05.2023
